# Geometric Sequence Triplets
A geometric sequence triplet is a sequence of three numbers where each successive number is obtained by multiplying the preceding number by a constant called the common ratio.

Let's examine three triplets to understand how this works:
- (1, 2, 4): This is a geometric sequence with a ratio of 2 (i.e., [1, 1⋅2 = 2, 2⋅2 = 4]).
- (5, 15, 45): This is a geometric sequence with a ratio of 3 (i.e., [5, 5⋅3 = 15, 15⋅3 = 45]).
- (2, 3, 4): Not a geometric sequence.

Given an array of integers and a common ratio r, find all triplets of indexes (i, j, k) that follow a geometric sequence for i < j < k. It's possible to encounter duplicate triplets in the array.

**Example:**<br/>
Input: nums = [2, 1, 2, 4, 8, 8], r = 2<br/>
Output: 5

Explanation:
- Triplet [2, 4, 8] occurs at indexes (0, 3, 4), (0, 3, 5), (2, 3, 4), (2, 3, 5).
- Triplet [1, 2, 4] occurs at indexes (1, 2, 3).

## Intuition
For a triplet to form a geometric sequence, it must adhere to two main rules:
1. It consists of three values that follow a geometric sequence with a common ratio `r`.
2. The three values forming the triplet must appear in the same order within the array as they do in the geometric sequence. This means that for a geometric triplet `(nums[i], nums[j], nums[k])`, the indices must follow the order `i < j < k`.

How can we represent a geometric sequence to satisfy rule 1? A triplet in a geometric sequence can be represented as `(x, x*r, x*r^2)`.

A brute force approach would iterate over all possible triplets in the array to check if any of them follow a geometric progression. However, this would require three nested loops to search through all triplets, resulting in a time complexity of **O(n³)**, where `n` denotes the length of the input array.

An important observation is that if we know one value of a triplet, we can calculate what the other two values should be.

However, we might encounter issues when using this triplet representation. While it's clear that `x*r` and `x*r^2` must be positioned to the right of `x`, we must ensure that the order of these values is maintained. To work around this issue, we use the representation `(x/r, x, x*r)`, which allows us to always maintain order by looking for `x/r` to the left of `x` and `x*r` to the right.

One way to find the `x/r` and `x*r` values is by linearly searching through the left and right subarrays. Performing this linear search for each number in the array results in an **O(n²)** time complexity. Can we improve it further?

---

### Using Hash Maps
A hash map is a great way to solve this problem, as it allows us to query specific values in constant time.
We need two hash maps:
- A hash map that tracks numbers to the left of each `x` (`left_map`).
- A hash map that tracks numbers to the right of each `x` (`right_map`).

Hash maps allow us to query both `x/r` in `left_map` and `x*r` in `right_map` in constant time on average. A hash map is preferred over a hash set because hash maps can store the frequency of each value they contain. This is crucial since the array might have duplicates, and we need to know the frequency of each value to accurately count all possible triplets.

---

### Finding All `(x/r, x, x*r)` Triplets
Before searching for `x/r`, we need to check if `x` is divisible by `r`. If it's not, it's impossible to form a valid triplet with the current `x`. Otherwise, we can proceed to look for the triplet.

For any element `x`, there could be multiple instances of `x/r` in `left_map` and multiple instances of `x*r` in `right_map`, implying that multiple triplets can be formed using `x` as the middle value. To get the total number of triplets with `x` in the middle, multiply the frequencies of `x/r` and `x*r`.

To summarize:
1. Check if `x % r == 0`.
2. Retrieve the frequency of `x/r` from `left_map` and `x*r` from `right_map`.
3. Multiply the frequencies to get the number of triplets with `x` as the middle value.

If either `x/r` or `x*r` is not found in its respective hash map, its frequency is `0` by default.

---

### Example
To ensure the hash maps always contain the correct values, we need to dynamically update them as we traverse the array. This is because the values in both hash maps change depending on the position of `x` in the array.

Since we traverse the array from left to right, we initially populate the `right_map` with all values in the array. This is because, before the start of the iteration, every element is a potential candidate for `x*r`. Meanwhile, `left_map` starts empty since there are no preceding elements to consider as potential `x/r` values.

Now, let’s look for triplets by considering each value as the middle value (`x`) of a triplet.
1. Update `right_map`: Remove the current value from `right_map` since this occurrence of `x` is no longer to the right of itself. If multiple instances exist, decrement its frequency.
2. Check if `x/r` is an integer. If it is, find the number of triplets with `x` as the middle number by multiplying the frequencies of `x/r` and `x*r` from the hash maps. If `x/r` is not present in `left_map`, its frequency is `0`.
3. Add the current value to `left_map` because it now becomes a potential `x/r` value for future triplets in the array.

Repeating this process for the entire array allows us to efficiently find all geometric triplets with a ratio of `r`. To clarify, the hash maps in each step represent their state at the current position of `x` in the array. This means that `left_map` includes values to the left of the current `x`, while `right_map` includes values to the right of it.

In [1]:
from typing import List
from collections import defaultdict

def geometric_sequence_triplets(nums: List[int], r: int) -> int:
    left_map = defaultdict(int)
    right_map = defaultdict(int)
    count = 0

    for x in nums:
        right_map[x] += 1

    for x in nums:
        right_map[x] -= 1
        if x % r == 0:
            count += left_map[x // r] * right_map[x * r]

        left_map[x] += 1

    return count

The time complexity is O(n) because we iterate through the nums array and perform constant-time hash map operations at each iteration.

The space complexity is O(n) because the hash maps can grow up to n in size.